<a href="https://colab.research.google.com/github/Tianarandr/python-for-datascience/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import collections
from typing import Optional
import numpy as np



class MulticlassSVM:
    """Class implementing a Support Vector Machine for multi-classification purposes based on one-vs-one strategy.
        Given N different classes to classify, the algorithm provides N*(N-1)/2 SVM binary classifiers.
        Each classifier is trained to correctly classify 2 of the N given classes using in the training process
        only the entries in the dataset to which it corresponds a label of the 2 classes.
        Given an unseen example, the prediction of the class is computed deploying a voting schema among the classifiers
    """

    def __init__(self,
                 kernel: Optional[str] = 'linear',
                 gamma: Optional[float] = None,
                 deg: Optional[int] = 3,
                 r: Optional[float] = 0.0,
                 C: Optional[float] = 1.):
        # self.SVMs: list of tuples, each one of 3 elements: (SVM_binary_classifier, 1st_class_label, 2nd_class_label)
        #            1st_class_label corresponds to sign "-", 2nd_class_label to sign "+"
        # Note: the number of binary SVM classifiers needed will be known only when the dataset labels will be given
        self.SVMs = []
        # By default linear kernel is used
        self.kernel = kernel
        # If gamma is None, it will be computed during fit process
        self.gamma = gamma
        self.deg = deg
        self.r = r
        self.C = C
        self.labels = None
        self.support_vectors = set()

    def fit(self, X: np.ndarray, y: np.ndarray):
        # check if labels are integers
        labels = np.unique(y)
        for label in labels:
            if not label.is_integer():
                raise ValueError(str(label) + " is not an integer value label")
        self.labels = np.array(labels, dtype=int)

        # re-arrange training set per labels in a dictionary
        X_arranged_list = collections.defaultdict(list)
        for index, x in enumerate(X):
            X_arranged_list[y[index]].append(x)

        # convert to numpy array the previous dictionary
        X_arranged_numpy = {}
        for index in range(len(self.labels)):
            X_arranged_numpy[index] = np.array(X_arranged_list[index])

        for i in range(0, self.labels.shape[0] - 1):
            for j in range(i + 1, self.labels.shape[0]):
                current_X = np.concatenate((X_arranged_numpy[i], X_arranged_numpy[j]))
                current_y = np.concatenate((- np.ones((len(X_arranged_numpy[i]),), dtype=int),
                                           np.ones(len((X_arranged_numpy[j]),), dtype=int)))
                svm = SVM(kernel=self.kernel, gamma=self.gamma, deg=self.deg, r=self.r, C=self.C)
                svm.fit(current_X, current_y, verbosity=0)
                for sv in svm.sv_X:
                    self.support_vectors.add(tuple(sv.tolist()))
                svm_tuple = (svm, self.labels[i], self.labels[j])
                self.SVMs.append(svm_tuple)
        print('{0:d} support vectors found out of {1:d} data points'.format(len(self.support_vectors), len(X)))

    def predict(self, X: np.ndarray):
        voting_schema = np.zeros([len(X), 2, self.labels.shape[0]], dtype=float)
        for svm_tuple in self.SVMs:
            prediction = svm_tuple[0].project(X)
            for i in range(len(prediction)):
                if prediction[i] < 0:
                    voting_schema[i][0][svm_tuple[1]] += 1
                    voting_schema[i][1][svm_tuple[1]] += -1 * prediction[i]
                else:
                    voting_schema[i][0][svm_tuple[2]] += 1
                    voting_schema[i][1][svm_tuple[2]] += prediction[i]

        voting_results = np.zeros(len(voting_schema), dtype=int)
        for i in range(len(voting_schema)):
            sorted_votes = np.sort(voting_schema[i][0])
            if sorted_votes[0] > sorted_votes[1]:
                voting_results[i] = voting_schema[i][0].argmax()
            else:
                voting_results[i] = voting_schema[i][1].argmax()

        return voting_results

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target


MulticlassSVM.fit(X, y)
predictions = clf.predict(X)

In [ ]:

   
import numpy as np


class SVM:
    def __init__(self, learning_rate=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        y_ = np.where(y <= 0, -1, 1)

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    self.w -= self.lr * (
                        2 * self.lambda_param * self.w - np.dot(x_i, y_[idx])
                    )
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


# Testing
if __name__ == "__main__":
    # Imports
    from sklearn import datasets
    import matplotlib.pyplot as plt

    X, y = datasets.make_blobs(
        n_samples=50, n_features=2, centers=2, cluster_std=1.05, random_state=40
    )
    y = np.where(y == 0, -1, 1)

    clf = SVM()
    clf.fit(X, y)
    # predictions = clf.predict(X)

    print(clf.w, clf.b)

    def visualize_svm():
        def get_hyperplane_value(x, w, b, offset):
            return (-w[0] * x + b + offset) / w[1]

        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        plt.scatter(X[:, 0], X[:, 1], marker="o", c=y)

        x0_1 = np.amin(X[:, 0])
        x0_2 = np.amax(X[:, 0])

        x1_1 = get_hyperplane_value(x0_1, clf.w, clf.b, 0)
        x1_2 = get_hyperplane_value(x0_2, clf.w, clf.b, 0)

        x1_1_m = get_hyperplane_value(x0_1, clf.w, clf.b, -1)
        x1_2_m = get_hyperplane_value(x0_2, clf.w, clf.b, -1)

        x1_1_p = get_hyperplane_value(x0_1, clf.w, clf.b, 1)
        x1_2_p = get_hyperplane_value(x0_2, clf.w, clf.b, 1)

        ax.plot([x0_1, x0_2], [x1_1, x1_2], "y--")
        ax.plot([x0_1, x0_2], [x1_1_m, x1_2_m], "k")
        ax.plot([x0_1, x0_2], [x1_1_p, x1_2_p], "k")

        x1_min = np.amin(X[:, 1])
        x1_max = np.amax(X[:, 1])
        ax.set_ylim([x1_min - 3, x1_max + 3])

        plt.show()

    visualize_svm()